In [ ]:
%matplotlib inline
from skimage import io, transform
import matplotlib.pyplot as plt
import numpy as np
from keras.applications.inception_v3 import InceptionV3
from keras.applications.imagenet_utils import decode_predictions
import tensorflow as tf

In [ ]:
image = io.imread('images/input/Maula.jpg')
target_image_size = (299, 299)
image = transform.resize(image, target_image_size)
plt.imshow(image)

In [ ]:
fig = plt.gcf()
DPI = fig.get_dpi()
fig.set_size_inches(299.0/float(DPI),299.0/float(DPI))
plt.imshow(image)
plt.axis('off')
fig.savefig('images/output/maula_299.png')

In [ ]:
model = InceptionV3()

In [ ]:
image = (image - 0.5)*2

In [ ]:
preds = model.predict(image[np.newaxis, ...])
idx = np.argmax(preds[0])
print(idx)
print(preds[0][idx])
print(decode_predictions(preds))

In [ ]:
with tf.GradientTape() as tape:
    inputs = image[np.newaxis, ...] #adding axis (batch)
    inputs_tf = tf.convert_to_tensor(inputs, dtype=tf.float32)
    tape.watch(inputs_tf) # Start watching this part
    preds = model(inputs_tf) # Make a prediction
    top_pred_index = tf.argmax(preds[0]) #Get the class of the highest probability
    top_class_channel = preds[:, top_pred_index]

In [ ]:
grads = tape.gradient(top_class_channel, inputs_tf)
grads_abs = tf.math.abs(grads)

In [ ]:
grads_max = tf.math.reduce_max(grads, axis=-1)

In [ ]:
fig = plt.gcf()
DPI = fig.get_dpi()
fig.set_size_inches(299.0/float(DPI),299.0/float(DPI))
plt.imshow(grads_max[0], vmin=np.min(grads_max[0]), vmax=np.max(grads_max[0]), alpha=0.5, cmap='RdBu')
plt.axis('off')
#fig.savefig('images/output/maula_saliency.png')